In [24]:
import time
import openpyxl
import pandas as pd
import tkinter as tk
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import requests
import json

def open_credamo(browser):
    browser.get("https://www.credamo.com")
    # 检测登录
    login = False
    for i in range(5):
        try:
            # # show_execute_info("请登录Credamo账号。")
            browser.find_element_by_class_name("login").click()
            time.sleep(1)
            for cookie in browser.get_cookies():
                if cookie['name']=='credamo-dms-auth':
                    login=True
                    break
            # print("未检测到登录，请登录Credamo账号。")
        except:
            time.sleep(1)
            pass
            # print("已登陆。")
    if login==True:
        # show_execute_info("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
        print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    else:
        # show_execute_info("请登录账号，然后打开一个项目。")
        print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    # show_execute_info("准备好excel后点击自动拒绝被试")
    print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    
def get_surveyId(browser):
    url = browser.current_url
    try:
        # 获得surveyId
        startloc = url.index("surveyId")+len("surveyId=")
        endloc = url.index("#")
        surveyId = url[url.index("surveyId")+len("surveyId="):url.index("#")]
    except:
        print("还没有打开项目,读取不到surveyId,请打开一个项目")
        # show_execute_info("还没有打开项目,读取不到surveyId,请打开一个项目")
        return 0
    
    #检测是否打开数据清理页面，如果没有则自动点击清理
    try:
        browser.find_element_by_class_name("iconfont icon-shaixuan")
        # 如果找到，表明已经打开数据清理页面
        print("已打开数据清理页面")
        # show_execute_info("已打开数据清理页面")
    except:    
    # 如果报错，没有打开数据清理页面，自动点击“数据清理”
        try:
            browser.find_element_by_class_name("iconfont icon-shaixuan")
        except:
            browser.find_element_by_link_text("数据清理").click()
            time.sleep(3)
    
    return int(surveyId)

def set_page_size(browser,page_size):
    page_sizes = [5,10,30,50,100]
    try:
        size_choise = page_sizes.index(page_size)
    except:
        print("没有该条数/页的选项，请检查")
        # show_execute_info("没有该条数/页的选项，请检查")
        return -2
    # 设置100页
    set_size = False
    retry_time = 0
    while (not set_size) and retry_time <= 5:
        try:
            browser.find_element_by_class_name('el-input__inner').click()
            time.sleep(1)
            pageSizes = browser.find_elements_by_class_name('el-select-dropdown__item')
            pageSizes[size_choise].click()
            page_size = page_sizes[size_choise]
            set_size = True
            return page_size
        except:
            print("设置条数/页失败，正在重试")
            # show_execute_info("设置条数/页失败，正在重试")
            retry_time += 1
    print("设置页码失败，请检查网页。")
    # show_execute_info("设置页码失败，请检查网页。")

def go_to_page(browser,page):
    try:
        loc = browser.find_element_by_class_name('el-pagination__jump').find_element_by_class_name('el-input__inner')
        loc.send_keys(Keys.CONTROL+'a')     #全选
        loc.send_keys(Keys.DELETE)		# 删除，清空
        loc.send_keys(page)	# 写入新的值
        loc.send_keys(Keys.ENTER)
        return page
    except:
        print("无法跳转页面")
        # show_execute_info("无法跳转页面")
        return 0

def set_encoding(browser):
    time.sleep(1)
    browser.find_element_by_link_text("数据清理").click()
    time.sleep(2)
    # delete some click that cause bugs 20231210
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[1]/div[@class='route']/div[@class='button2']").click()
    # time.sleep(1)
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[@class='resultView']/div[@class='search-box']/div[@class='popover'][4]").click()
    # time.sleep(1)
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[1]/div[@class='route']/div[@class='button1']").click()
    # time.sleep(1)

def get_page_df(browser,surveyId,page_size,page_num,download=False):
    try:
        time.sleep(1)
        cookies = browser.get_cookies()
        cookie_str = ""
        for cookie in cookies:
            cookie_str = "{}={}; ".format(cookie['name'],cookie['value']) + cookie_str
        cookie_str = cookie_str[:-2]
        
        headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
                    "referer": "https://www.credamo.com/survey.html?surveyId=%d}" % surveyId,
                    "cookie":cookie_str}
        url = "https://www.credamo.com/v1/cleanVar/qstOverviewBySurId/%d?currPageSize=%d&currPageIndex=%d"  % (surveyId,page_size,page_num)
    except:
        print("获取cookie失败，请重试。")
        # show_execute_info("获取cookie失败，请重试。")
        return []
    try:
        resp = requests.get(url=url,headers=headers)
        if resp.status_code == 200:
            output = json.loads(resp.content.decode('utf8'))
            data = output['data']
            ques_headers = data['header']
            total_cnt = output['total']
            ques_headers_kv = {}
            for header in ques_headers:
                ques_headers_kv[header["id"]] = header['qNum']+" "+header['questionName']
            df = pd.DataFrame(output['data']['rowList'])
            #如果下载，则不替换列名称，最后再替换
            if not download:
                df.columns = [ques_headers_kv[id] if id not in ['status', 'answerId', 'userId'] else id for id in df.columns]
                return df,total_cnt
            else:
                return df,total_cnt,ques_headers_kv,ques_headers
        else:
            print("无法登录到问卷页面")
            # show_execute_info("无法登录到问卷页面")
            return pd.DataFrame([]),0
    except:
        print("无法get问卷页面")
        # show_execute_info("无法get问卷页面")
        return pd.DataFrame([]),0

def download_data(browser,download_name):
    surveyId = get_surveyId(browser)
    set_encoding(browser)
    page_size = set_page_size(browser,page_size=10)
    page_num = 1
    page_num = go_to_page(browser,page_num)
    page_df,total_cnt,ques_headers_kv,ques_headers = get_page_df(browser,surveyId,page_size,page_num,download=True)
    total_page_num = total_cnt//page_size+1
    page_dfs = pd.DataFrame([])
    for page_num in range(total_page_num):
        page_num += 1
        print(page_num)
        # go_to_page(browser,page_num)
        # time.sleep(5)
        page_df,total_cnt,ques_headers_kv,_ = get_page_df(browser,surveyId,page_size,page_num,download=True)
        if 'userId' in page_df.columns:
            page_df.set_index('userId')
            if str(page_dfs.shape)=="(0, 0)":
                page_dfs = page_df
            else:
                page_dfs = pd.concat([page_dfs,page_df],ignore_index=True)
    page_dfs.columns = [
        ques_headers_kv[id] 
        if id not in ['status', 'answerId', 'userId'] 
        else id 
        for id in page_dfs.columns
    ]
    page_dfs = page_dfs[['status', 'answerId', 'userId'] + [ques_headers_kv[ques_header['id']] for ques_header in ques_headers]]
    page_dfs = page_dfs[(page_dfs=="*").sum(axis=1)==0] #剔除拒绝的
    download_name_str = download_name.get('0.0','end').strip()+"_"+time.strftime('%Y%m%d_%H%M')+".xlsx"
    page_dfs.to_excel(download_name_str)
    # show_execute_info("{}下载完成".format(download_name_str))

# def get_userId_loc(excel_name,page_df,page_num,autobatchReject):    
#     chosen_ids = excel_name.get("1.0","end").strip().split('\n')
#     print('指定拒绝的被试:',chosen_ids)
    
#     page_userIds = page_df['userId'].tolist()
#     print(page_df.shape,page_df.columns)
#     print('page_userIds',page_userIds)
#     page_chosen_ids = list(set(chosen_ids) & set(page_userIds))
#     print('page_chosen_ids')
#     chosen_rows = []
#     for chosen_id in page_chosen_ids:
#         print('page_userIds',page_userIds)
#         chosen_row = page_userIds.index(chosen_id)+1
#         chosen_rows.append(chosen_row)
#         print('chosen_row:', chosen_row)
#         # change to relative xpath 20231227
#         xpath_expression = f"//table[@style='margin-top: 0px; width: 415px;']/tbody/tr[{chosen_row}]/td[1]/div/span"
#         browser.find_element_by_xpath(xpath_expression).click()
#     # 批量拒绝
#     browser.find_element_by_class_name("el-dropdown").click() # 点击批量操作
#     time.sleep(1)
#     if autobatchReject == True:
#         browser.find_elements_by_class_name("el-dropdown-menu__item")[1].click() # 点击拒绝选中数据
#         time.sleep(1)
#         el_select = browser.find_elements_by_class_name("el-select")[1].click()
#         msgbox = browser.find_element_by_class_name("el-message-box")
#         msgbox.click()
#         print("请根据网页提示进行操作")
#         time.sleep(3)
#         msgbox.find_element_by_class_name("el-button").click()
#         print("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
#         # show_execute_info("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
#         time.sleep(3)
#     return chosen_rows 
def get_userId_loc(excel_name,page_df,page_num,autobatchReject):    
    chosen_ids = excel_name.get("1.0","end").strip().split('\n')
    print('指定拒绝的被试:',chosen_ids)
    
    page_userIds = page_df['userId'].tolist()
    print(page_df.shape,page_df.columns)
    print('page_userIds',page_userIds)
    page_chosen_ids = list(set(chosen_ids) & set(page_userIds))
    print('page_chosen_ids')
    chosen_rows = []
    for chosen_id in page_chosen_ids:
        print('page_userIds',page_userIds)
        chosen_row = page_userIds.index(chosen_id)+1
        chosen_rows.append(chosen_row)
        print('chosen_row:', chosen_row)
        # change to relative xpath 20231227
        xpath_expression = f"//table[@style='margin-top: 0px; width: 415px;']/tbody/tr[{chosen_row}]/td[1]/div/span"
        browser.find_element_by_xpath(xpath_expression).click()
    # 批量拒绝
    browser.find_element_by_class_name("el-dropdown").click() # 点击批量操作
    time.sleep(1)
    if autobatchReject == True:
        browser.find_elements_by_class_name("el-dropdown-menu__item")[1].click() # 点击拒绝选中数据
        time.sleep(3)
        try:
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='el-select' and @style='width: 380px;']"))).click()  # 点击请选择
        except TimeoutException:
            ok_button_locator = (By.XPATH, "//button[@class='el-button el-button--default el-button--small el-button--primary ']/span[contains(text(), '确定')]")
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable(ok_button_locator)).click()
            # show_execute_info("第%d页无拒绝被试" % page_num)
            print("第%d页无拒绝被试" % page_num)
            time.sleep(3)
            return []

        reject_reason_li_locator = (By.XPATH, "//li[contains(@class, 'el-select-dropdown__item') and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable(reject_reason_li_locator)).click()

        reject_button_locator = (By.XPATH, "//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable(reject_button_locator)).click()
        # el_select = browser.find_element_by_xpath("//div[@class='el-select' and @style='width: 380px;']") # 点击请选择
        # el_select.click()
        # time.sleep(3)
        # reject_reason_li = browser.find_element_by_xpath("//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
        # reject_reason_li.click()
        # time.sleep(3)
        # reject_button = browser.find_element_by_xpath("//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
        # reject_button.click()
        # time.sleep(3)
        # show_execute_info("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
        print("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
        time.sleep(3)
    return chosen_rows 


def batchReject(browser,excel_name,page_num_assign_text,autoReject=True):
    surveyId = get_surveyId(browser)
    page_size = set_page_size(browser,page_size=10)
    page_num = 1
    page_num = go_to_page(browser,page_num)
    page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
    total_page_num = total_cnt//page_size+1
    if autoReject:
        for page_num in range(total_page_num):
            page_num += 1
            # print(page_num)
            go_to_page(browser,page_num)
            time.sleep(5)
            page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
            # print(total_cnt)
            userId_loc = get_userId_loc(excel_name,page_df,page_num,autoReject)
            # print(userId_loc)
        # show_execute_info('所有页面已拒绝。\n你可以打开另一个项目，清空原用户ID后粘贴新项目待拒绝用户ID，重新点击拒绝被试。\n或者关闭浏览器，关闭本程序')
    else:
        #手动拒绝
        try:
            page_num_assign = int(page_num_assign_text.get('0.0','end').strip())
        except:
            print("无法读取页面数，请在上方小方框中输入正确的页面数字")
            # show_execute_info("无法读取页面数，请在上方小方框中输入正确的页面数字")
            return 0
        if page_num_assign <= 0 or page_num_assign > total_cnt//page_size+1:
            print("输入的页面数字超过答卷范围，请检查页面数")
            # show_execute_info("输入的页面数字超过答卷范围，请检查页面数")
        go_to_page(browser,page_num_assign)
        time.sleep(5)
        page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num_assign)
        # print(total_cnt)
        userId_loc = get_userId_loc(excel_name,page_df,page_num_assign,autoReject)
        # print(userId_loc)
        print('页面%d已选中待拒绝被试，可以手动在浏览器中点击拒绝。\n' % page_num_assign)
        # show_execute_info('页面%d已选中待拒绝被试，可以手动在浏览器中点击拒绝。\n'%page_num_assign)

# def show_execute_info(var,cmd=False):
#     if cmd:
#         print(var)
#     else:
#         execute_info.insert("0.0", '\n' + time.strftime('%Y-%m-%d %H:%M:%S ') + var + '\n')

In [2]:
browser = webdriver.Chrome()

In [7]:
open_credamo(browser)

已登陆，请在浏览器中手动打开您需要拒绝被试的项目。
已登陆，请在浏览器中手动打开您需要拒绝被试的项目。


In [25]:
window = tk.Tk()
excel_name = tk.Text(window, height=10)
excel_name.pack()

In [26]:
#
excel_name.insert("0.0", 
"""
qM6z95xM
RMWkBWPM
ve8zb5z7
5LwjvYq7
kM14Bl2e
p7YZgnbL
""")

In [27]:
excel_name.get("1.0","end").strip().split('\n')

['qM6z95xM', 'RMWkBWPM', 've8zb5z7', '5LwjvYq7', 'kM14Bl2e', 'p7YZgnbL']

In [28]:
batchReject(
    browser=browser,
    excel_name=excel_name,
    page_num_assign_text=1,
    autoReject=True
)

指定拒绝的被试: ['qM6z95xM', 'RMWkBWPM', 've8zb5z7', '5LwjvYq7', 'kM14Bl2e', 'p7YZgnbL']
(10, 56) Index(['作答时长 作答时长(秒)',
       'Q3_1_20 请认真阅读以下题目，并根据题目要求作答。这是问卷最后部分。谢谢！(请您按照题目要求，并根据自己的实际情况作答。-您过去是否购买双肩包)',
       'Q3_1_22 请认真阅读以下题目，并根据题目要求作答。这是问卷最后部分。谢谢！(请您按照题目要求，并根据自己的实际情况作答。-您认为您的作答数据可信程度是？1 = 极不可信, 7 = 极可信)',
       'Q3_1_21 请认真阅读以下题目，并根据题目要求作答。这是问卷最后部分。谢谢！(请您按照题目要求，并根据自己的实际情况作答。-您的婚姻状态)',
       '屏幕分辨率 屏幕分辨率',
       'Q1_1_2 请您根据以下题目要求，根据自己的真实情况作答。(请下拉选择或填空。-如果患上某种疾病的概率是10%，那么1000人中有多少人预期会患上这种疾病？请输入数字)',
       'Q1_1_1 请您根据以下题目要求，根据自己的真实情况作答。(请下拉选择或填空。-您现在处于安静的环境么？如果不是, 请转移到安静的环境。)',
       '省份 省份',
       'Q1_1_3 请您根据以下题目要求，根据自己的真实情况作答。(请下拉选择或填空。-如果5个人的彩票号码一样，都中奖了，总奖金是2000000，那么，可以分得多少钱？请输入数字)',
       '发布ID 发布ID', '纬度 纬度', '设备类型 设备类型', '经度 经度', '结束时间 结束时间',
       'Q3_1_11 请认真阅读以下题目，并根据题目要求作答。这是问卷最后部分。谢谢！(请您按照题目要求，并根据自己的实际情况作答。-请输入一个数字评价上述商品的质量（从1到100，0表示极差；100表示极好）)',
       'Q3_1_10 请认真阅读以下题目，并根据题目要求作答。这是问卷最后部分。谢谢！(请您按照题目要求，并根据自己的实际情况作答。-这家网店正以很大的折扣出售上述商品。（1：非常不同意，7：非常同意）)',
       '

ValueError: not enough values to unpack (expected 2, got 0)

In [41]:
el_select = browser.find_element_by_xpath("//div[@class='el-select' and @style='width: 380px;']") # 点击请选择
el_select.click()

In [15]:
reject_reason_li = browser.find_element_by_xpath("//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
reject_reason_li

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]"}
  (Session info: chrome=109.0.5414.120)


In [23]:
reject_reason_li = browser.find_element_by_xpath("//li[contains(@class, 'el-select-dropdown__item') and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
reject_reason_li.click()

In [45]:
len(reject_reasons)

10

In [19]:

dropdown__items = browser.find_elements_by_class_name("el-select-dropdown__item")

In [22]:
dropdown__item[-1].click()

In [49]:
reject_button = browser.find_elements_by_xpath("button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
reject_button

[]

In [50]:
reject_button = browser.find_element_by_xpath("//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
reject_button

<selenium.webdriver.remote.webelement.WebElement (session="93ed257b692f814b704b82fdb1c162f2", element="212a5120-41f8-4d60-96d3-bf275ed1e5d1")>

In [23]:
ok_button = browser.find_element_by_xpath("//button[@class='el-button el-button--default el-button--small el-button--primary ']/span[contains(text(), '确定')]")
ok_button.click()

In [ ]:
<div role="dialog" aria-modal="true" aria-label="拒绝问卷" class="el-dialog refuse_dialog_bg" style="margin-top: 15vh; width: 500px;"><div class="el-dialog__header"><span class="el-dialog__title">拒绝问卷</span><button type="button" aria-label="Close" class="el-dialog__headerbtn"><i class="el-dialog__close el-icon el-icon-close"></i></button></div><div class="el-dialog__body"><div data-v-3e6f8ccc="" class="refuse_form"><div data-v-3e6f8ccc="" style="width: 70px;">拒绝原因: </div> <div data-v-3e6f8ccc="" class="new_css"><div data-v-3e6f8ccc="" class="el-select" style="width: 380px;"><!----><div class="el-input el-input--suffix"><!----><input type="text" readonly="readonly" autocomplete="off" placeholder="请选择" class="el-input__inner"><!----><span class="el-input__suffix"><span class="el-input__suffix-inner"><i class="el-select__caret el-input__icon el-icon-arrow-up"></i><!----><!----><!----><!----><!----></span><!----></span><!----><!----></div></div></div></div> </div><div class="el-dialog__footer"><span data-v-3e6f8ccc="" class="dialog-footer"><button data-v-3e6f8ccc="" type="button" class="el-button el-button--default"><!----><!----><span>取消</span></button> <!----> <button data-v-3e6f8ccc="" type="button" class="el-button el-button--primary"><!----><!----><span>批量拒绝</span></button></span></div></div>

In [ ]:
        reject_reason_li_locator = (By.XPATH, "//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable(reject_reason_li_locator)).click()